# Lattice Dynamics and Thermal Conductivity Tutorial
This notebook demonstrates how to compute the lattice thermal conductivity of PbTe using lattice dynamics and a machine-learning-based neuroevolution potential (NEP). It covers both the finite displacement method (FDM) and the temperature-dependent effective potential (TDEP) method for extracting interatomic force constants (IFCs).

## Required Dependencies
- ASE (Atomic Simulation Environment)
- HiPhive
- GPUMD
- Phonopy
- Phono3py
- calorine
- trainstation (for regression fitting)

## File Structure and Purpose
- `POSCAR`: Relaxed primitive cell (VASP format)
- `SPOSCAR`: Supercell for IFCs fitting (VASP format)
- `model.xyz`: Structure file for MD simulations
- `nep.txt`: NEP potential file (machine-learned, generated by training)
- `run.in`: GPUMD input file for MD or FDM simulations
- `training_structures.extxyz`: Displacement-force dataset for fitting IFCs
- `fc2.hdf5`, `fc3.hdf5`: Fitted second- and third-order force constants for Phono3py

## 1. NEP Potential Training (if not provided)
If you do not have a NEP potential file (`nep.txt`), you need to train it using your dataset. The training command depends on the NEP package you use. Example (replace with your NEP tool):
```bash
neptune train input.json
# or use your NEP training tool and dataset
```
After training, place the generated `nep.txt` in this directory.

## 2. Finite Displacement Method (FDM) Workflow
Generate randomly displaced structures and build the displacement-force dataset using HiPhive and ASE.

In [ ]:
from ase.io import read, write
from hiphive.structure_generation import generate_mc_rattled_structures
from calorine.calculators import CPUNEP
# Parameters
n_structures = 100
rattle_std = 0.02
minimum_distance = 2.0
prim = read('/opt/GPUMD/examples/25/POSCAR', format='vasp')
atoms_ideal = read('/opt/GPUMD/examples/25/POSCAR', format='vasp')
calc = CPUNEP('/opt/GPUMD/examples/25/nep.txt')
structures_mc = generate_mc_rattled_structures(atoms_ideal, n_structures, rattle_std, minimum_distance)
training_structures = []
for atoms in structures_mc:
    atoms.calc = calc
    training_structures.append(atoms)
write('training_structures.extxyz', training_structures)
print('Total number of displaced configurations:', len(training_structures))

## 3. Fit Interatomic Force Constants (IFCs) with HiPhive
Fit second- and third-order IFCs using the displacement-force dataset.

In [ ]:
from hiphive import ClusterSpace, StructureContainer, ForceConstantPotential
from trainstation import Optimizer
prim = read('/opt/GPUMD/examples/25/POSCAR')
atoms_ideal = read('/opt/GPUMD/examples/25/SPOSCAR')
structures = read('training_structures.extxyz', index=':')
cutoffs = [8.0, 6.0]
cs = ClusterSpace(prim, cutoffs)
sc = StructureContainer(cs)
for structure in structures:
    sc.add_structure(structure)
opt = Optimizer(sc.get_fit_data())
opt.train()
fcp = ForceConstantPotential(cs, opt.parameters)
fcp.write('fcc-pbte-highT.fcp')
fcs = fcp.get_force_constants(atoms_ideal)
fcs.write_to_phonopy('FORCE_CONSTANTS_fdm_2nd', format='text')
fcs.write_to_shengBTE('FORCE_CONSTANTS_fdm_3rd', prim)
fcs.write_to_phonopy('fc2.hdf5')
fcs.write_to_phono3py('fc3.hdf5')

## 4. Compute Lattice Thermal Conductivity with Phono3py
After fitting IFCs, use Phono3py to compute lattice thermal conductivity. Run the following command in the shell:
```bash
phono3py --dim 5 5 5 --mesh 12 12 12 --br --fc2 --fc3
```
- `--dim 5 5 5`: Supercell dimensions
- `--mesh 12 12 12`: q-point mesh
- `--br`: Use relaxation time approximation
- `--fc2 --fc3`: Use fitted second- and third-order IFCs

## 5. TDEP Method Workflow (Temperature-Dependent IFCs)
Use GPUMD to generate MD trajectories and build a temperature-dependent displacement-force dataset.

In [ ]:
# Example GPUMD run.in for TDEP dataset generation
run_in_content = '''
potential nep.txt
time_step       1
velocity        300
ensemble        nvt_nhc 300 300 100
dump_thermo     1000
run             50000
ensemble        nve
time_step       1
dump_thermo     1000
dump_exyz       100 0 1
run             10000
'''
with open('run.in', 'w') as f:
    f.write(run_in_content)

After running GPUMD with this input, use the generated `dump.xyz` to fit temperature-dependent IFCs with HiPhive as above.